In [2]:
%matplotlib inline
%config InlineBackend.figure_format = "retina"
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import scipy.special as sc
from haloclustering import statistics as stats
import pickle
from astropy.cosmology import Planck15 as cosmo

In [4]:
def log_probability():
    None

pkl_file = '../exclusion-model/exclusion_model_gauss_1h_sampler.pkl'
infile = open(pkl_file, "rb")
sampler = pickle.load(infile)
infile.close()

pkl_file = '../exclusion-model/model_2h_w_beta_sampler.pkl'
infile = open(pkl_file, "rb")
sampler_2h = pickle.load(infile)
infile.close()

In [5]:
# get data
df = pd.read_csv('../data/cgm2_casbah_velo_split_table_with_comoving_rho_and_mass.csv')
df['log_rho'] = np.log10(df.rho)
df.sort_values('rho', inplace=True)
df.rho = df.rho / 1000 # convert to Mpc
df = df.query("(HM_0_500 != 'indeterminate') & (7 < mstars < 12)")
df['Hz'] = cosmo.H(df.z).value
vmax = 500.0 # km/s

df.dropna(subset=['mstars'], inplace=True)
massmask = df.eval("8 < mstars < 10.5").values
df.mstars = 10**df.mstars

outcomes = df.HM_0_500
hits = (outcomes.values == "hit") & massmask
misses = (outcomes.values == "miss") & massmask

In [6]:
# calculate the likelihoods 
data_args = (df.rho.values, df.z.values, df.Hz.values, vmax, df.mstars.values, hits, misses)
params_chain = sampler.get_chain(discard=2125, thin=1, flat=True)
params = np.nanmean(params_chain, axis=0)
params_2h_chain = sampler_2h.get_chain(discard=2000, thin=1, flat=True)
params_2h = np.nanmean(params_2h_chain, axis=0)
ll_exc = stats.log_likelihood_exc(*params, *data_args)
ll_2h = stats.log_likelihood_2h(*params_2h, *data_args)

3.603113933775093 1.5462836768996562 0.07666961135741135


In [7]:
# calculate the BICs
bic_exc = stats.bayes_inf_criterion(ll_exc[0], len(params), len(df.z.values))
bic_2h = stats.bayes_inf_criterion(ll_2h[0], len(params_2h), len(df.z.values))
bic_exc, bic_2h

(6821.547515661012, 6805.394688180214)

In [8]:
# calculate the AICs
aic_exc = stats.akaike_inf_criterion(ll_exc[0], len(params))
aic_2h = stats.akaike_inf_criterion(ll_2h[0], len(params_2h))
aic_exc, aic_2h

(6766.444085108986, 6770.955044085197)